In [1]:
from openai import OpenAI
from github import Github
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
client = OpenAI(
    api_key=os.getenv("GPT_KEY"),
)

In [3]:
def detect_broken_hierarchy(code):
    prompt = "Broken Hierarchy smell arises when a supertype and its subtype conceptually do not share an IS-A relationship resulting in broken substitutability. Identify this smell in below code as the methods of super class are not used in sub class and also give the refactored version:\n" + code + "\n---\n"
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        return response
    except Exception as e:
        print("An error occurred:", e)
        return []

In [4]:
# Read code sample from file
file_path = "../books-web/src/main/java/com/sismics/books/rest/resource/ThemeResource.java"
with open(file_path, "r") as file:
    code_sample = file.read()

In [5]:
t1 = detect_broken_hierarchy(code_sample)

In [6]:
print(t1)

ChatCompletion(id='chatcmpl-8vRJtQ8jKVOfCXwRjDbkukNVf8a9n', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='In the given code, the ThemeResource class extends BaseResource but does not use any of its methods. This indicates a broken hierarchy smell.\n\nTo refactor the code and remove the broken hierarchy smell, we can remove the inheritance from BaseResource if it is not needed. If there are common methods that need to be shared between ThemeResource and other resources, those methods can be extracted into a separate class and used as a composition instead of inheritance.\n\nRefactored code:\n```java\npackage com.sismics.books.rest.resource;\n\nimport com.sismics.books.core.dao.file.theme.ThemeDao;\nimport org.codehaus.jettison.json.JSONException;\nimport org.codehaus.jettison.json.JSONObject;\n\nimport javax.ws.rs.GET;\nimport javax.ws.rs.Path;\nimport javax.ws.rs.Produces;\nimport javax.ws.rs.core.MediaType;\nimport javax.ws.rs.cor

In [7]:
refactored = (t1.choices[0].message.content).split("```")[1][5:]
print(refactored)

package com.sismics.books.rest.resource;

import com.sismics.books.core.dao.file.theme.ThemeDao;
import org.codehaus.jettison.json.JSONException;
import org.codehaus.jettison.json.JSONObject;

import javax.ws.rs.GET;
import javax.ws.rs.Path;
import javax.ws.rs.Produces;
import javax.ws.rs.core.MediaType;
import javax.ws.rs.core.Response;
import java.util.ArrayList;
import java.util.List;

/**
 * Theme REST resources.
 * 
 * @author jtremeaux
 */
@Path("/theme")
public class ThemeResource {
    /**
     * Returns the list of all themes.
     * 
     * @return Response
     * @throws JSONException
     */
    @GET
    @Produces(MediaType.APPLICATION_JSON)
    public Response list() throws JSONException {
        ThemeDao themeDao = new ThemeDao();
        List<String> themeList = themeDao.findAll();
        JSONObject response = new JSONObject();
        List<JSONObject> items = new ArrayList<>();
        for (String theme : themeList) {
            JSONObject item = new JSONObject();
  

In [37]:
new_file = open(file_path, "w")
new_file.write(refactored)
new_file.close()

In [ ]:
github_token = os.getenv("GITHUB_TOKEN")
# print(github_token)

github_repo = "serc-courses/se-project-1--_23"

g = Github(github_token)
repo = g.get_repo(github_repo)

# Create a new branch
branch_name = "refactored_broken_hierarchy_ThemeResource"
repo.create_git_ref(ref='refs/heads/' + branch_name, sha=repo.get_branch("master").commit.sha)

In [11]:
new_file_path = "books-web/src/main/java/com/sismics/books/rest/resource/ThemeResource.java"
file_content = refactored

try:
    # Check if the file already exists
    file=repo.get_contents(new_file_path, ref=branch_name)
    repo.update_file(new_file_path, f"Update {new_file_path}", file_content, file.sha, branch_name)
    print(f"File '{file_path}' updated successfully.")
except Exception as e:
    # If the file does not exist, create it
    repo.create_file(new_file_path, f"Add {new_file_path}", file_content, branch_name)
    print(f"File '{new_file_path}' added successfully.")

File '../books-web/src/main/java/com/sismics/books/rest/resource/ThemeResource.java' updated successfully.


In [12]:
pull_request = repo.create_pull(
    title="Fixed Broken Hierarchy in ThemeResource.java",
    body="Solution given by ChatGPT",
    head=branch_name,
    base="bonus"
)

print("Pull request created:", pull_request.html_url)

Pull request created: https://github.com/serc-courses/se-project-1--_23/pull/24
